In [ ]:
import ROOT
from T2KStyle.T2KStyle import GetT2K
style = GetT2K(3)
%jsroot on
ROOT.gROOT.SetStyle("T2K")
c = ROOT.TCanvas('c', '', 800, 600)
cc = ROOT.TCanvas('cc', '', 800, 600)
ccc = ROOT.TCanvas('ccc', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

# Z scan

## MC

In [ ]:
path = '/eos/experiment/neutplatform/np07/HAT/MC/ana_results/v9/'
ver = 'v9_'
x_axis = [10, 20, 30, 40, 50, 60, 70, 80, 90]
file_mc = [['412 ns', [ROOT.TFile(path + f'/z90_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z80_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z70_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z60_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z50_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z40_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z30_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z20_{ver}iter1.root'),
                       ROOT.TFile(path + f'/z10_{ver}iter1.root')],
#                        ROOT.TFile(path + f'/z95_{ver}iter1.root')],
            [(x - (100 - 97.96)) for x in x_axis]
           ]
          ]

x_title = 'Drift [cm]'

## DATA

In [ ]:
path = '/eos/experiment/neutplatform/np07/HAT/DESY_2021/ana_output/z_scan2/'
volt = '275'
time = '412'
it = '9'
x_axis = [860, 760, 660, 560, 460, 360, 260, 160, 60, -40]
file_name = [['412 ns', 
             [ROOT.TFile(path + f'/z_{time}ns_{volt}V_860_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_760_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_660_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_560_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_460_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_360_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_260_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_160_iter{it}.root'),
              ROOT.TFile( path + f'/z_{time}ns_{volt}V_60_iter{it}.root'),
              ROOT.TFile(path + f'/z_{time}ns_{volt}V_m40_iter{it}.root')]
             ],
             [(x + 92.36) for x in x_axis]
            ]
title = ['860 mm', '760 mm', '660 mm', '560 mm', '460 mm', '360 mm', '260 mm', '160 mm', '60 mm', '-40 mm']


color = [ROOT.kRed, ROOT.kGreen, ROOT.kBlack, ROOT.kBlue, ROOT.kViolet, ROOT.kYellow, ROOT.kCyan, ROOT.kOrange, ROOT.kSpring, ROOT.kAzure]

# Plotting

### Merge all residuals and do joint fit

In [ ]:
c.cd()
gr = []
opt = 'ap'
for num_main, file_list in enumerate(file_mc):
    gr.append(ROOT.TGraphErrors())
    for num, file in enumerate(file_list[1]):
        h = ROOT.TH1F(f'h_{num}', '', 200, -3., 3.)
        print(file.GetName())
        file.outtree.Project(f'h_{num}', 'residual*1e3', 'abs(residual) < 0.01')
        h.Fit('gaus', 'Q')
        s = h.GetFunction('gaus').GetParameter(2)
#         print(file_list[2][num], s*1000, sep='\t')
        gr[-1].SetPoint(gr[-1].GetN(), file_list[2][num], s*1000)
        gr[-1].SetPointError(gr[-1].GetN() - 1, 0, h.GetFunction('gaus').GetParError(2) * 1000)
        
gr[0].GetYaxis().SetRangeUser(0., 3500)
gr[0].GetXaxis().SetRangeUser(-10., 100)
for num_main, gr_one in enumerate(gr):
    if num_main > 0: opt = 'p same'
    gr_one.Draw(opt)
    gr_one.SetMarkerColor(1+num_main)
    gr_one.SetLineColor(1+num_main)
    gr_one.SetTitle(f'{file_mc[num_main][0]};{x_title}; Resolution [#mum];')
c.Draw()
c.BuildLegend()

### Fit clusters independently

In [ ]:
c.cd()
func = ROOT.TF1('func', 'sqrt([0]*[0] + [1]*x)', 0, 1000)
func.SetParName(0, '#sigma_{0}')
func.SetParName(1, 'C_{diff}^{2}/N_{eff}')
opt = 'ap'
gr = []
for num_main, file_list in enumerate(file_mc):
    gr.append(ROOT.TGraphErrors())
    for num, file in enumerate(file_list[1]):
        mean = 0
        v = []
        for col in range(1, 70):
            h = ROOT.TH1F(f'h_{num}_{col}', '', 200, -3., 3.)
            file.outtree.Project(f'h_{num}_{col}', 'residual*1e3', f'abs(residual) < 0.1 && pad_x == {col}')
            if h.Integral() < 100: continue
            h.Fit('gaus', 'Q')
            if not h.GetFunction('gaus') or h.GetFunction('gaus').GetParameter(2) > 5:
                continue
            mean += h.GetFunction('gaus').GetParameter(2)
            v.append(h.GetFunction('gaus').GetParameter(2))
        if not len(v):
            continue
        rms = 0
        mean /= len(v)
        for val in v:
            rms += (val - mean)**2
        rms = rms**0.5 / len(v)
#         print(file_list[2][num], mean/10, sep='\t')
        
        gr[-1].SetPoint(gr[-1].GetN(), file_list[2][num], mean /10 )
        gr[-1].SetPointError(gr[-1].GetN() - 1, 0, rms /10 )

gr[0].GetYaxis().SetRangeUser(0., 0.25)
gr[0].GetXaxis().SetRangeUser(-10., 100)
for num_main, gr_one in enumerate(gr):
    if num_main > 0: opt = 'p same'
    gr_one.Draw(opt)
    gr_one.SetMarkerColor(1+num_main)
    gr_one.SetLineColor(1+num_main)
    gr_one.SetTitle(f'{file_mc[num_main][0]};{x_title}; Resolution [cm];')

    ROOT.gStyle.SetOptFit(1)
#     gr_one.Fit('func')
c.Draw()
c.BuildLegend()